In [29]:
import csv
import pysam

In [30]:
#why you write so many lines???

# namefile = open('/home/local/ARCS/hq2130/missense/data/reformat.cnn.csv_list.txt')
# line = 'start~'
# list = []
# while line:
#     line = namefile.readline()
#     if line == '':
#         break
#     print line
#     print line[-1]
#     list.append('/home/local/ARCS/hq2130/missense/data/'+line[2:-1])
# #print list
# list.append('/home/local/ARCS/hq2130/missense/data/output/output_data_mode5.csv')

namefile = '/home/local/ARCS/hq2130/missense/data/reformat.cnn.csv_list.txt'
with open(namefile) as f:
    lst = ['/home/local/ARCS/hq2130/missense/data/' + line.strip()[2:] for line in f.readlines()]
fins = ['/home/local/ARCS/hq2130/missense/data/output/output_data_mode5.csv']

fins = ['../data/case_control/case_MarkDaly.anno.rare.HIS.reformat.cnn.csv',
        '../data/case_control/case_MarkDaly.anno.rare.HS.reformat.cnn.csv',
        '../data/case_control/control_MarkDaly.anno.rare.HIS.reformat.cnn.csv',
        '../data/case_control/control_MarkDaly.anno.rare.HS.reformat.cnn.csv']

In [31]:
def GetScoreREVEL(Chr, Pos, Ref, Alt, Tabix):
    for row in Tabix.fetch(Chr, int(Pos)-1, int(Pos)+1):# 0-based inputin .fetch
        row = row.split('\t')
        if row[3] == Ref and row[4] == Alt:
            return row[5]
    return 'NA'

## orginal bed format .txt needs to be bgzip then tabix -
REVEL = '/data/hq2130/large_files/revel_file/revel_all_chr.txt.gz' 
f_revel= pysam.TabixFile(REVEL)

def GetScoreMPC(Chr, Pos, Ref, Alt, Tabix):
    for row in Tabix.fetch(Chr, int(Pos)-1, int(Pos)+1):# 0-based inputin .fetch
        row = row.split('\t')
        if row[2] == Ref and row[3] == Alt:
            return row[-1]
    return 'NA'

MPC = '/data/hq2130/large_files/fordist_constraint_official_mpc_values.txt.gz' 
f_MPC = pysam.TabixFile(MPC)

In [34]:

fins = ['../data/case_control/case_MarkDaly.anno.rare.csv']
for S in fins:   
    print S
    fin = open(S, 'rb')
    fout = open(S+'Ann.mpc.csv','wb')
    writer = csv.writer(fout)
    reader = csv.reader(fin) # a list
    header = reader.next()
    header.extend(['REVEL', 'MPC'])
    writer.writerow(header)
    idx_chr = header.index('hg19_chr')
    idx_pos = header.index('hg19_pos(1-based)')
    idx_ref = header.index('ref')
    idx_alt = header.index('alt')
    for l in reader:
        Chr = l[idx_chr]
        Pos = l[idx_pos]
        Ref = l[idx_ref]
        Alt = l[idx_alt]
        revel_score = GetScoreREVEL(Chr, Pos, Ref, Alt, f_revel)
        mpc_score = GetScoreMPC(Chr, Pos, Ref, Alt, f_MPC)
#         if revel_score == 'NA':
#             print ','.join([Chr,Pos,Ref,Alt])
        l.extend([revel_score, mpc_score])
        writer.writerow(l)
    fin.close()
    fout.close()

../data/case_control/case_MarkDaly.anno.rare.csv


In [ ]:
## this is a debugging test 06082017
## the bug is due to incomplete download of revel_all_chr.txt T_T
import pysam
import csv
REVELtxt = '/data/hq2130/large_files/revel_file/revel_all_chr.txt' ## orginal bed format .txt needs to be bgzip then tabix -
newchrnum = 0
fin = open(REVELtxt, 'rU')
#fout = open(S+'Ann.csv','wb')
#writer = csv.writer(fout)
reader = csv.reader(fin) # a list
header = reader.next()
print header
#header.extend(['REVEL'])
for line in reader:
    #print line
    chr = line[0].split()[0]
    if newchrnum != chr:
        print chr
        newchrnum = chr  
fin.close()
'''
REVEL = '/data/hq2130/large_files/revel_file/revel_all_chr.txt.gz' ## orginal bed format .txt needs to be bgzip then tabix -
f_revel= pysam.TabixFile(REVEL)
for row in f_revel.fetch(3, 10000, 10001):
    print row
for row in f_revel.fetch(4, 10000, 10001):
    print row
'''

In [ ]:
## this is a debugging test 06092017
## the bug is (chr,pos-1, pos) should be (chr,pos-1,pos+1)

import csv
import pysam

REVEL = '/data/hq2130/large_files/revel_file/revel_all_chr.txt.gz' ## orginal bed format .txt needs to be bgzip then tabix -
f_revel= pysam.TabixFile(REVEL)

S9 = ['/home/local/ARCS/hq2130/missense/data/metaSVM/metaSVM_train.anno.rare.csv']

def GetScoreREVEL(Chr, Pos, Ref, Alt, Tabix):
	for row in Tabix.fetch(Chr, int(Pos)-1, int(Pos)):# 0-based inputin .fetch
		row = row.split('\t')
		if row[3] == Ref and row[4] == Alt:
			return row[5]
	return 'NA'

num = 0
for S in S9:   
	fin = open(S, 'rb')
	fout = open(S+'_cc_new_Ann.csv','wb')
	writer = csv.writer(fout)
	reader = csv.reader(fin) # a list
	header = reader.next()
	header.extend(['REVEL'])
	writer.writerow(header)
	idx_chr = header.index('hg19_chr')
	idx_pos = header.index('hg19_pos(1-based)')
	idx_ref = header.index('ref')
	idx_alt = header.index('alt')
	for l in reader:
		if num == 1:
			break
		num +=1  
		Chr = l[idx_chr]
		Pos = l[idx_pos]
		Ref = l[idx_ref]
		Alt = l[idx_alt]
		print Chr
		print Pos
		revel_score = GetScoreREVEL(Chr, Pos, Ref, Alt, f_revel)		
		print revel_score
		#mcap_score = GetScoreMCAP(Chr, Pos, Ref, Alt, f_mcap)		
		l.extend([revel_score])
		writer.writerow(l)
	fin.close()
	fout.close()

print 'printing a test1:'
print Chr, Pos
print f_revel.fetch(Chr,int(Pos)-1,int(Pos)+1)
print 'printing a test2:'
for row in f_revel.fetch(Chr,int(Pos)-1,int(Pos)+1):### shenme gui!!!! 0-based inputin .fetch
        row = row.split('\t')       
        print row
        if row[3] == Ref and row[4] == Alt:
            print row[5]

#test under unix
#tabix revel_all_chr.txt.gz 1:1139498-1139499
    